# Blog: Getting Started with LangChain: A Beginner’s Guide to Building LLM-Powered Applications
- https://towardsdatascience.com/getting-started-with-langchain-a-beginners-guide-to-building-llm-powered-applications-95fc8898732c

In [15]:
# import os
# os.environ["OPENAI_API_KEY"]='<Type API Key' 

# Basic LLM

In [16]:
# Proprietary LLM from e.g. OpenAI
# pip install openai
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")


# The LLM takes a prompt as an input and outputs a completion
prompt = "Alice has a parrot. What animal is Alice's pet?"
completion = llm(prompt)

In [5]:
completion

"\n\nAlice's pet is a parrot."

In [45]:
llm

OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f8648a68f70>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False)

# Embedding

In [6]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# The embeddings model takes a text as an input and outputs a list of floats
text = "Alice has a parrot. What animal is Alice's pet?"
text_embedding = embeddings.embed_query(text)

In [9]:
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', document_model_name='text-embedding-ada-002', query_model_name='text-embedding-ada-002', embedding_ctx_length=-1, openai_api_key=None, chunk_size=1000, max_retries=6)

In [10]:
print("dimentsion size: ", len(text_embedding))

dimentsion size:  1536


In [11]:
print("a little bit of dimentsion : ", text_embedding[0:10])

a little bit of dimentsion :  [0.013288114219903946, -0.009326402097940445, -0.008071756921708584, -0.011409236118197441, -0.013473529368638992, 0.011149654164910316, -0.005432675592601299, 0.0032571330666542053, -0.0020812894217669964, -0.0018170722760260105]


# Prompts: Managing LLM inputs

## Zero-shot pormpt

In [18]:
from langchain import PromptTemplate

template = "What is a good name for a company that makes {product}?"

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

# few-shot prompt

In [19]:
from langchain import PromptTemplate, FewShotPromptTemplate

examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

example_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n",
)

few_shot_prompt.format(input="big")

'Give the antonym of every input\n\nWord: happy\nAntonym: sad\n\n\n\nWord: tall\nAntonym: short\n\n\nWord: big\nAntonym:'

# Chains: Combining LLMs 

In [22]:
print("prompt: \n", prompt)

prompt: 
 input_variables=['product'] output_parser=None partial_variables={} template='What is a good name for a company that makes {product}?' template_format='f-string' validate_template=True


In [21]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, 
                  prompt = prompt)

# Run the chain only specifying the input variable.
chain.run("colorful socks")

'\n\nHappy Socks & Co.'

## Chaining Two LLMs
- Prompt (Product) --> LLM(Product):Compnay Name -> LLM(Company Name): Catchphrase --> Catchphrase
- 흐름
    - Prompt 제공 
        - What is a good name for a company that makes {colorful socks}
    - 첫번째 LLM 으로 부터 회사 이름 답변 받음
        - Rainbow Toes
    - 두번째 prompt 제공 (위의 회사 이름을 입력으로 제공)
        - Write a catchphrase for the following company: {Rainbow Toes} 
    - 두번째 LLM 으로 부터 catchphrase 답변 받음
        - "Put a little rainbow in your step!"
        

In [25]:
from langchain.chains import LLMChain, SimpleSequentialChain

# Define the first chain as in the previous code example
# ...

# Create a second chain with a prompt template and an LLM
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Combine the first and the second chain 
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("colorful socks")
print("catchphrase: \n", catchphrase)



> Entering new SimpleSequentialChain chain...


Rainbow Toes.


"Put a little rainbow in your step!"

> Finished chain.
catchphrase: 
 

"Put a little rainbow in your step!"


# Indexes: Accessing external data

In [28]:
! pip install youtube-transcript-api --quiet
! pip install pytube --quiet


![youtube_Next_js13.png](img/youtube_Next_js13.png)

In [30]:

from langchain.document_loaders import YoutubeLoader

# loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
loader = YoutubeLoader("_w0Ikk4JY7U", True)
    
documents = loader.load()

In [31]:
documents

[Document(page_content="it is October 26 2022 and you're watching the code report just when you thought you had web development all figured out Along Comes an entirely new build tool and a complete overhaul of your favorite framework we're ready for a new paradigm not enough men's and disease of evil the reality [Music] JavaScript it's a wild ride next JS comp happened yesterday and they made an extremely ballsy move to take the world's most popular JavaScript framework and linchpin of a unicorn startup and totally change up the developer experience as you're about to see these new features are game changing and blazingly fast but they're not without their trade-offs buckle up because we've got a lot to unpack starting with a brand new build tool called turbo pack now I know you just switched over to the next gen build tool V last week but a new character is spawned versel employs Tobias coppers the creator of webpack who knows a thing or two about module bundling turbo pack was built 

In [33]:
# pip install faiss-cpu
from langchain.vectorstores import FAISS

# create the vectorestore to use as the index
db = FAISS.from_documents(documents, embeddings)


하나의 문서가 저장이 되어 있음

In [36]:
db.index_to_docstore_id

{0: '55c2880c-7075-4b9e-9d09-8b5f471adfe7'}

### 위의 임베딩 정보를 가지고 Question & Answer 를 함.

In [35]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True)

query = "What am I never going to do?"
result = qa({"query": query})

print(result['result'])

 You are never going to write data similar to remix forms in the new app directory.


# Memory: Remembering previous conversations

In [39]:
from langchain import ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Alice has a parrot.")

conversation.predict(input="Bob has two cats.")

conversation.predict(input="How many pets do Alice and Bob have?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Alice has a parrot.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Alice has a parrot.
AI:  That's interesting. What kind of parrot does Alice have?
Human: Bob has two cats.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provi

' It looks like Alice has one pet, a parrot, and Bob has two pets, two cats.'

# Agents: Accessing other tools

In [43]:
# ! pip install wikipedia --quiet

In [44]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)


agent.run("When was Barack Obama born? How old was he in 2022?")



> Entering new AgentExecutor chain...
 I should look up information about Barack Obama.
Action: Wikipedia
Action Input: Barack Obama
Observation: Page: Barack Obama
Summary: Barack Hussein Obama II ( (listen) bə-RAHK hoo-SAYN oh-BAH-mə; born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American  president of the United States. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer before holding public office. 
Obama was born in Honolulu, Hawaii. After graduating from Columbia University in 1983, he worked as a community organizer in Chicago. In 1988, he enrolled in Harvard Law School, where he was the first black president of the Harvard Law Review. After graduating, he became a civil rights attorney and an academic, teaching constitutional la

'Barack Obama was born on August 4, 1961, and was 61 years old in 2022.'